<a href="https://colab.research.google.com/github/bhend449/ML-RTS-detection-model-builder/blob/master/RTS_Detection_Model_EI2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#RTS Detection Modeler

This creates a model to detect bi-stable RTS signals from a large array of unknown signals.

###Mount to Google Drive

Assume the files are stored on a private Google Drive folder

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


###Check location of files 

In [0]:

!ls "/content/gdrive/My Drive/Colab data files"

model.png  RTSvecTrainMicroScaled.mat  WNvecTrainMicroScaled.mat


###Import .mat files with SciPi and prepare the training and testing sets


In [0]:
import matplotlib
import matplotlib.pyplot as plt
import h5py
import numpy as np
import scipy.io
'''
        Import two vectors, one RTS and another white noise
'''

mat = scipy.io.loadmat("/content/gdrive/My Drive/Colab data files/WNvecTrainMicroScaled.mat")
WNvec = mat["ScaledSet"]   # Must be called the same as the variable it was saved 
                    # under in Matlab (or however the block was created)
WNvec = np.array(WNvec)
#WNvec = np.transpose(WNvec)

mat = scipy.io.loadmat("/content/gdrive/My Drive/Colab data files/RTSvecTrainMicroScaled.mat")
RTSvec = mat["RTSdata"]   # Must be called the same as the variable it was saved 
                    # under in Matlab (or however the block was created)
RTSvec = np.array(RTSvec)
'''
        Plot a few to double check you're working with the correct data
'''
'''
x = np.arange(0,1500)
for i in range(9000,10000):
        p = WNvec[i,:]
        plt.plot(x,p)
        plt.savefig('C:/Users/Ben WORK ONLY/Desktop/GH repos/RTS ML detect beta/%d' % (i))
        plt.close()
'''
'''
        Randomize the order of columns(?) and combine data sets
'''

np.random.shuffle(RTSvec)
np.random.shuffle(WNvec)

RTStrain,RTStest = RTSvec[:80000,:],RTSvec[80000:,:]
WNtrain,WNtest = WNvec[:80000,:],WNvec[80000:,:]


x_train = np.concatenate((RTStrain,WNtrain),axis=0)
x_test = np.concatenate((RTStest,WNtest),axis=0)

y_train = np.zeros(160000)
y_train[80000:]=1
y_test = np.zeros(20000)
y_test[10000:]=1

     
del RTStest
del RTStrain
del RTSvec
del WNtest
del WNtrain
del WNvec

###Inspect the variables

In [0]:
variables = %who_ls
for v in variables:
    print(v, type(eval(v)).__name__, str(eval(v)))


print('x_train = ',np.shape(x_train))
print('x_test = ',np.shape(x_test))
print('y_train = ',np.shape(y_train))
print('y_test = ',np.shape(y_test))


Activation type <class 'keras.layers.core.Activation'>
Conv1D type <class 'keras.layers.convolutional.Conv1D'>
Dense type <class 'keras.layers.core.Dense'>
Dropout type <class 'keras.layers.core.Dropout'>
Embedding type <class 'keras.layers.embeddings.Embedding'>
Flatten type <class 'keras.layers.core.Flatten'>
GlobalAveragePooling1D type <class 'keras.layers.pooling.GlobalAveragePooling1D'>
LSTM type <class 'keras.layers.recurrent.LSTM'>
MaxPooling1D type <class 'keras.layers.pooling.MaxPooling1D'>
Sequential type <class 'keras.engine.sequential.Sequential'>
X_test ndarray [[[0.33514777]
  [0.17056111]
  [0.21287812]
  ...
  [0.65930081]
  [0.74429395]
  [0.74751007]]

 [[0.19815345]
  [0.37758127]
  [0.21647565]
  ...
  [0.08159848]
  [0.2374728 ]
  [0.46649323]]

 [[0.47576634]
  [0.44945879]
  [0.3874687 ]
  ...
  [0.70991637]
  [0.50191965]
  [0.65678079]]

 ...

 [[0.49994813]
  [0.43655318]
  [0.32192905]
  ...
  [0.58576659]
  [0.52918915]
  [0.76833364]]

 [[0.5223964 ]
  [0.3

### Define the structure of the model



In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D, Flatten, LSTM
import numpy as np

X_train = np.expand_dims(x_train, axis=2) # reshape (569, 30) to (569, 30, 1) 
X_test = np.expand_dims(x_test, axis=2) # reshape (569, 30) to (569, 30, 1) 

model = Sequential()
model.add(Conv1D(32, 12, activation='relu', input_shape=(1500, 1)))
model.add(MaxPooling1D(3))
model.add(Dropout(0.5))
model.add(Conv1D(64, 12, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Dropout(0.5))
model.add(Conv1D(128, 12, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Using TensorFlow backend.


### Fit the model and Score

In [0]:
model.fit(X_train, y_train, batch_size=16, epochs=2)
score = model.evaluate(X_test, y_test, batch_size=16)

Epoch 1/2
160000/160000 [==============================] - 972s 6ms/step - loss: 0.2826 - acc: 0.8888
Epoch 2/2
20000/20000 [==============================] - 34s 2ms/step


In [0]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install -q pydot && pip3 install ann_visualizer
import pydot

  Running setup.py bdist_wheel for ann-visualizer ... - done
  Stored in directory: /root/.cache/pip/wheels/b6/b4/4e/d92f50c9c4f004cf315a0e0fcd455486bd799c50fe80cf1f5d
Successfully built ann-visualizer


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1489, 32)          416       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 496, 32)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 496, 32)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 485, 64)           24640     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 161, 64)           0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 161, 64)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 150, 128)          98432     
__________

In [0]:
from keras.utils import plot_model
plot_model(model, to_file='/content/gdrive/My Drive/Colab data files/model_EI2019.png')